In [1]:
import json
import requests
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [2]:
stocks = ['GOOGL', 'META', 'AAPL', 'NFLX', 'AMZN']

product_data = {
    'url': "https://www.alphavantage.co/query?",
    'functions': ['TIME_SERIES_INTRADAY', 'TIME_SERIES_DAILY'],
    'intraday_interval': '15min',
    'output_size': 'compact',
    'products': ['Shopify', 'IBM'],
    'symbols': ['SHOP.TRT', 'IBM'],
    'APIKey': '7NVAEAZYAXPYKMZB'
}
print(product_data['url'])

#READ Data on Daily Time Series for Shopify - Symbol: SHOP.TRT - API-KEY: 7NVAEAZYAXPYKMZB
TSIurl = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=GOOGL&interval=30min&outputsize=full&apikey=7NVAEAZYAXPYKMZB'
response = requests.get(TSIurl)
timeseries_intraday = response.json()
StockURL = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol=GOOGL&apikey=7NVAEAZYAXPYKMZB'
stocksdata = requests.get(StockURL).json()
#print(timeseries_intraday['Time Series (30min)'].keys())
#print(stocksdata['Name'])

#Create Database Connection
def server_connection(host_name, username, pwd):
    connection = None
    try:
        connection = mysql.connector.connect(host = host_name, user = username, passwd = pwd)
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

server = server_connection('localhost', 'root', 'abcd')

#Create a new database
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")
        
create_database(server, "CREATE DATABASE SOEN6441")

In [3]:
def create_db_connection(host_name, username, pwd, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(host = host_name, user = username, passwd = pwd, database = db_name)
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

connection = create_db_connection('localhost', 'root', '#Snroshan1998', "SOEN6441")


MySQL Database connection successful


In [4]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

create_table_stocks = """CREATE TABLE STOCKS (  STOCKNAME VARCHAR(1000),
                                                SYMBOL VARCHAR(20) PRIMARY KEY,
                                                SECTOR VARCHAR(1000),
                                                INDUSTRY VARCHAR(2000),
                                                MARKETCAP BIGINT,
                                                ASSETTYPE VARCHAR(300),
                                                COMPANY_DESCRIPTION VARCHAR(10000)
                                                )"""

execute_query(connection, create_table_stocks)

#Execute a given Query
    
stockinfo = """INSERT INTO STOCKS VALUES ('%s', '%s', '%s', '%s', '%d', '%s', "%s")""" % (stocksdata['Name'], stocksdata['Symbol'], stocksdata['Sector'], stocksdata['Industry'], int(stocksdata['MarketCapitalization']), stocksdata['AssetType'], stocksdata['Description'])
#print(stockinfo)

execute_query(connection, stockinfo)


def add_stocks(stocks):
    for stock in stocks:
        url = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol=%s&apikey=7NVAEAZYAXPYKMZB' % (stock)
        stock_data = requests.get(url).json()
        stock_query= """INSERT INTO STOCKS VALUES ('%s', '%s', '%s', '%s', '%d', '%s', "%s")""" % (stock_data['Name'], stock_data['Symbol'], stock_data['Sector'], stock_data['Industry'], int(stock_data['MarketCapitalization']), stock_data['AssetType'], stock_data['Description'])
        
        execute_query(connection, stock_query)

add_stocks(stocks)


#ID INT AUTO_INCREMENT PRIMARY KEY, 
def create_stock_tables(stocks):
    for stock in stocks:
        create_table = """CREATE TABLE %s ( ID INT PRIMARY KEY,
                                            STOCKSYMBOL VARCHAR(20),
                                            DATENTIME DATETIME NOT NULL,
                                            OPEN FLOAT(9, 4),
                                            HIGH FLOAT(9, 4),
                                            LOW FLOAT(9, 4),
                                            CLOSE FLOAT(9, 4)
                                            )""" % (stock)

        execute_query(connection, create_table)

        
create_stock_tables(stocks)

In [5]:
def get_values(stock, interval, data):
    intraday_values = []
    for datetime in data:
        intraday_values.append([datetime, float(data[datetime]['1. open']), float(data[datetime]['2. high']), float(data[datetime]['3. low']), float(data[datetime]['4. close'])])
    return (intraday_values)

def update_intraday_values(stocks, interval = '15min'):
    for stock in stocks:
        #execute_query(connection, """DELETE FROM %s""" %(stock))
        url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&interval=%s&symbol=%s&outputsize=full&apikey=7NVAEAZYAXPYKMZB' %(interval, stock)
        intraday_data = requests.get(url).json()
        values = get_values(stock, interval, intraday_data['Time Series (%s)' %(interval)])
        closing_data = get_hourly_data(stock, interval, intraday_data['Time Series (%s)' %(interval)])
            #intraday_update_query= """INSERT INTO %s VALUES (%s, '%s', '%s', %f, %f, %f, %f)""" % (stock, n, intraday_data['Meta Data']['2. Symbol'], i[0], i[1], i[2], i[3], i[4])
        #print (closing_data)
        #for n, i in zip(range(1, len(values) + 1), values):
            
            #execute_query(connection, intraday_update_query)
            #execute_query(connection, intraday_update_query)
        #supply values into the query update above and pass into function to execute query
        
                                                                                           
update_intraday_values(stocks)

#UPDATE INTRADAY VALUES FOR ALL STOCKS
def update_intraday_values(stocks, interval = '15min'):
    for stock in stocks:
        execute_query(connection, """DELETE FROM %s""" %(stock))
        url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&interval=%s&symbol=%s&outputsize=full&apikey=7NVAEAZYAXPYKMZB' %(interval, stock)
        intraday_data = requests.get(url).json()
        values = get_values(stock, interval, intraday_data['Time Series (%s)' %(interval)])
        for n, i in zip(range(1, len(values) + 1), values):
            intraday_update_query= """INSERT INTO %s VALUES (%s, '%s', '%s', %f, %f, %f, %f)""" % (stock, n, intraday_data['Meta Data']['2. Symbol'], i[0], i[1], i[2], i[3], i[4])
            execute_query(connection, intraday_update_query)
        #supply values into the query update above and pass into function to execute query
        
                                                                                           
update_intraday_values(stocks)

In [37]:
#OBTAIN DAILY DATA
def update_daily_data(connection, stocks, interval = '15min'):
    for stock in stocks:
        cursor = connection.cursor()
        daily_data = {}
        
        get_dates = """SELECT DISTINCT DATE(DATENTIME) FROM {0} WHERE DATENTIME LIKE "%20:00:00" """.format(stock)
        cursor.execute(get_dates)
        result = cursor.fetchall()

        for i in range(len(result)-1):
            open_value = read_query(connection, """SELECT CLOSE FROM {0} WHERE DATENTIME LIKE "{1}_20:00:00" """.format(stock, result[i+1][0]))[0][0]
            close_value = read_query(connection, """SELECT CLOSE FROM {0} WHERE DATENTIME LIKE "{1}_20:00:00" """.format(stock, result[i][0]))[0][0]
            daily_data[result[i][0]] = {"Open": open_value, "High": '', "Low": '', "Close": close_value}

      
        
        
        
    
update_daily_data(connection, stocks)

{datetime.date(2022, 11, 3): {'Open': 87.04, 'High': '', 'Low': '', 'Close': 83.68}, datetime.date(2022, 11, 2): {'Open': 90.58, 'High': '', 'Low': '', 'Close': 87.04}, datetime.date(2022, 11, 1): {'Open': 94.8, 'High': '', 'Low': '', 'Close': 90.58}, datetime.date(2022, 10, 31): {'Open': 96.33, 'High': '', 'Low': '', 'Close': 94.8}, datetime.date(2022, 10, 28): {'Open': 92.0, 'High': '', 'Low': '', 'Close': 96.33}, datetime.date(2022, 10, 27): {'Open': 95.35, 'High': '', 'Low': '', 'Close': 92.0}, datetime.date(2022, 10, 26): {'Open': 97.6001, 'High': '', 'Low': '', 'Close': 95.35}, datetime.date(2022, 10, 25): {'Open': 102.9, 'High': '', 'Low': '', 'Close': 97.6001}, datetime.date(2022, 10, 24): {'Open': 101.22, 'High': '', 'Low': '', 'Close': 102.9}, datetime.date(2022, 10, 21): {'Open': 98.09, 'High': '', 'Low': '', 'Close': 101.22}, datetime.date(2022, 10, 20): {'Open': 99.58, 'High': '', 'Low': '', 'Close': 98.09}, datetime.date(2022, 10, 19): {'Open': 102.44, 'High': '', 'Low': 

In [8]:
#OBTAIN HOURLY DATA
def update_hourly_data(connection, stocks, interval = '15min'):
    for stock in stocks:
        cursor = connection.cursor()
        hourly_data = {}
        
        get_hour = """SELECT DISTINCT DATENTIME FROM {0} WHERE DATENTIME LIKE "%:00:00" """.format(stock)
        get_interval = """SELECT DISTINCT DATENTIME FROM {0} WHERE DATENTIME LIKE "%:00" """.format(stock)
        cursor.execute(get_interval)
        result = cursor.fetchall()
        for i in range(0, 4):
            print(result[i][0])
        #print(str(result[0][0]).split()[0])

        for i in range(len(result)-1):
            open_value = read_query(connection, """SELECT CLOSE FROM {0} WHERE DATENTIME LIKE "{1}" """.format(stock, result[i+1][0]))[0][0]
            close_value = read_query(connection, """SELECT CLOSE FROM {0} WHERE DATENTIME LIKE "{1}" """.format(stock, result[i][0]))[0][0]
            high_value = read_query(connection, """SELECT MAX(HIGH) FROM {0} WHERE DATENTIME LIKE "{1}%" """.format(stock, str(result[0][0]).split()[0]))[0][0]
            low_value = read_query(connection, """SELECT MIN(LOW) FROM {0} WHERE DATENTIME LIKE "{1}%" """.format(stock, str(result[0][0]).split()[0]))[0][0]
            
            hourly_data[result[i][0]] = {"Open": open_value, "High": high_value, "Low": low_value, "Close": close_value}
    
    
    
    
        #print(stock, hourly_data)
       
        
update_hourly_data(connection, ['GOOGL', 'META', 'AAPL', 'AMZN'])

2022-11-03 20:00:00
2022-11-03 19:45:00
2022-11-03 19:30:00
2022-11-03 19:15:00
2022-11-03 20:00:00
2022-11-03 19:45:00
2022-11-03 19:30:00
2022-11-03 19:15:00
2022-11-03 20:00:00
2022-11-03 19:45:00
2022-11-03 19:30:00
2022-11-03 19:15:00
2022-11-03 20:00:00
2022-11-03 19:45:00
2022-11-03 19:30:00
2022-11-03 19:15:00


In [11]:
#OBTAIN HOURLY DATA
def update_hourly_data(connection, stocks, interval = '15min'):
    for stock in stocks:
        cursor = connection.cursor()
        hourly_data = {}
        
        get_hour = """SELECT DISTINCT DATENTIME FROM {0} WHERE DATENTIME LIKE "%:00:00" """.format(stock)
        get_interval = """SELECT DISTINCT DATENTIME FROM {0} WHERE DATENTIME LIKE "%:00" """.format(stock)
       
        cursor.execute(get_hour)
        result = cursor.fetchall()
        cursor.execute(get_interval)
        res1 = cursor.fetchall()
        
        #print(str(result[0][0]).split()[0])

        for i, j in zip(range(len(result)-1), range(4, len(res1)-1, 4)):
            high_value, low_value = [], []
            open_value = read_query(connection, """SELECT CLOSE FROM {0} WHERE DATENTIME LIKE "{1}" """.format(stock, result[i+1][0]))[0][0]
            close_value = read_query(connection, """SELECT CLOSE FROM {0} WHERE DATENTIME LIKE "{1}" """.format(stock, result[i][0]))[0][0]
            for hours in range(j-4, j):
#                intervals.append(res1[hours][0])
                high_value.append(read_query(connection, """SELECT HIGH FROM {0} WHERE DATENTIME LIKE "{1}" """.format(stock, res1[hours][0]))[0][0])
                low_value.append(read_query(connection, """SELECT LOW FROM {0} WHERE DATENTIME LIKE "{1}" """.format(stock, res1[hours][0]))[0][0])
            #print(high_value, low_value)
            hourly_data[result[i][0]] = {"Open": open_value, "High": max(high_value), "Low": min(low_value), "Close": close_value}

    
        print(hourly_data)
    
        
update_hourly_data(connection, ['GOOGL', 'META', 'AAPL', 'AMZN'])

{datetime.datetime(2022, 11, 3, 20, 0): {'Open': 83.69, 'High': 83.75, 'Low': 83.5, 'Close': 83.68}, datetime.datetime(2022, 11, 3, 19, 0): {'Open': 83.47, 'High': 83.76, 'Low': 83.5, 'Close': 83.69}, datetime.datetime(2022, 11, 3, 18, 0): {'Open': 83.66, 'High': 83.71, 'Low': 83.44, 'Close': 83.47}, datetime.datetime(2022, 11, 3, 17, 0): {'Open': 83.41, 'High': 83.7, 'Low': 83.41, 'Close': 83.66}, datetime.datetime(2022, 11, 3, 16, 0): {'Open': 84.27, 'High': 84.31, 'Low': 83.34, 'Close': 83.41}, datetime.datetime(2022, 11, 3, 15, 0): {'Open': 84.465, 'High': 84.55, 'Low': 84.0, 'Close': 84.27}, datetime.datetime(2022, 11, 3, 14, 0): {'Open': 85.3982, 'High': 85.43, 'Low': 84.39, 'Close': 84.465}, datetime.datetime(2022, 11, 3, 13, 0): {'Open': 84.71, 'High': 85.43, 'Low': 84.59, 'Close': 85.3982}, datetime.datetime(2022, 11, 3, 12, 0): {'Open': 85.78, 'High': 85.95, 'Low': 84.61, 'Close': 84.71}, datetime.datetime(2022, 11, 3, 11, 0): {'Open': 85.5, 'High': 85.99, 'Low': 85.075, 'Clo

{datetime.datetime(2022, 11, 3, 20, 0): {'Open': 88.76, 'High': 88.9, 'Low': 88.7, 'Close': 88.79}, datetime.datetime(2022, 11, 3, 19, 0): {'Open': 88.83, 'High': 88.98, 'Low': 88.74, 'Close': 88.76}, datetime.datetime(2022, 11, 3, 18, 0): {'Open': 88.85, 'High': 88.98, 'Low': 88.75, 'Close': 88.83}, datetime.datetime(2022, 11, 3, 17, 0): {'Open': 88.9, 'High': 89.08, 'Low': 88.73, 'Close': 88.85}, datetime.datetime(2022, 11, 3, 16, 0): {'Open': 89.645, 'High': 89.97, 'Low': 88.775, 'Close': 88.9}, datetime.datetime(2022, 11, 3, 15, 0): {'Open': 89.5465, 'High': 89.727, 'Low': 89.23, 'Close': 89.645}, datetime.datetime(2022, 11, 3, 14, 0): {'Open': 90.17, 'High': 90.29, 'Low': 89.05, 'Close': 89.5465}, datetime.datetime(2022, 11, 3, 13, 0): {'Open': 88.54, 'High': 90.4576, 'Low': 88.46, 'Close': 90.17}, datetime.datetime(2022, 11, 3, 12, 0): {'Open': 89.9799, 'High': 90.15, 'Low': 88.51, 'Close': 88.54}, datetime.datetime(2022, 11, 3, 11, 0): {'Open': 88.7925, 'High': 90.14, 'Low': 88.

{datetime.datetime(2022, 11, 3, 20, 0): {'Open': 137.92, 'High': 137.93, 'Low': 137.69, 'Close': 137.73}, datetime.datetime(2022, 11, 3, 19, 0): {'Open': 137.91, 'High': 137.96, 'Low': 137.4, 'Close': 137.92}, datetime.datetime(2022, 11, 3, 18, 0): {'Open': 139.12, 'High': 139.2, 'Low': 137.8, 'Close': 137.91}, datetime.datetime(2022, 11, 3, 17, 0): {'Open': 138.88, 'High': 139.23, 'Low': 138.5102, 'Close': 139.12}, datetime.datetime(2022, 11, 3, 16, 0): {'Open': 140.24, 'High': 140.33, 'Low': 138.75, 'Close': 138.88}, datetime.datetime(2022, 11, 3, 15, 0): {'Open': 139.53, 'High': 140.35, 'Low': 139.085, 'Close': 140.24}, datetime.datetime(2022, 11, 3, 14, 0): {'Open': 140.6426, 'High': 140.79, 'Low': 139.52, 'Close': 139.53}, datetime.datetime(2022, 11, 3, 13, 0): {'Open': 139.49, 'High': 140.77, 'Low': 139.27, 'Close': 140.6426}, datetime.datetime(2022, 11, 3, 12, 0): {'Open': 141.055, 'High': 141.134, 'Low': 139.26, 'Close': 139.49}, datetime.datetime(2022, 11, 3, 11, 0): {'Open': 

{datetime.datetime(2022, 11, 3, 20, 0): {'Open': 89.28, 'High': 89.35, 'Low': 89.2, 'Close': 89.31}, datetime.datetime(2022, 11, 3, 19, 0): {'Open': 89.13, 'High': 89.41, 'Low': 89.09, 'Close': 89.28}, datetime.datetime(2022, 11, 3, 18, 0): {'Open': 89.5775, 'High': 89.64, 'Low': 89.11, 'Close': 89.13}, datetime.datetime(2022, 11, 3, 17, 0): {'Open': 89.3, 'High': 89.62, 'Low': 89.12, 'Close': 89.5775}, datetime.datetime(2022, 11, 3, 16, 0): {'Open': 89.84, 'High': 90.23, 'Low': 89.25, 'Close': 89.3}, datetime.datetime(2022, 11, 3, 15, 0): {'Open': 89.8601, 'High': 90.005, 'Low': 89.36, 'Close': 89.84}, datetime.datetime(2022, 11, 3, 14, 0): {'Open': 90.9416, 'High': 91.1, 'Low': 89.7601, 'Close': 89.8601}, datetime.datetime(2022, 11, 3, 13, 0): {'Open': 89.335, 'High': 91.08, 'Low': 89.02, 'Close': 90.9416}, datetime.datetime(2022, 11, 3, 12, 0): {'Open': 90.795, 'High': 91.15, 'Low': 89.16, 'Close': 89.335}, datetime.datetime(2022, 11, 3, 11, 0): {'Open': 90.63, 'High': 91.14, 'Low':

In [6]:
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")


In [7]:
#EXTRACT HOURLY DATA FROM INTRADAY
def get_daily_data(stock, interval, intraday_data):
    #search = """SELECT DATENTIME, OPEN, CLOSE, HIGH, LOW FROM {0} WHERE DATENTIME LIKE "%20:00:00" """.format(stock)
    dates = """SELECT DISTINCT DATE(DATENTIME) FROM {0} WHERE DATENTIME LIKE "%20:00:00" """.format(stock)
    print(read_query(connection, dates))
    